# Lesson 1: Introduction to Deep Learning

In [1]:
import torch
import numpy as np

Напишите функцию `function01`. Она должна иметь следующую сигнатуру:

`def function01(tensor: torch.Tensor, count_over: str) -> torch.Tensor:`

Если `count_over` равен `'columns'`, верните среднее тензора по колонкам. Если равен `'rows'`, то верните среднее по рядам. Гарантируется, что тензор будет матрицей (то есть будет иметь размерность 2).  
Отправляемый файл должен иметь расширение **.py**

In [2]:
def function01(tensor: torch.tensor, count_over: str) -> torch.Tensor:
    if count_over == "columns":
        return torch.mean(tensor, dim=0) # dim=0 means columns dim=1 means rows
    else:
        return torch.mean(tensor, dim=1)


In [3]:
# let's create a tensor
t = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float32)

# calculate mean over columns
print(function01(t, "columns"))

# calculate mean over rows
print(function01(t, "rows"))

tensor([2., 5.])
tensor([2.5000, 3.5000, 4.5000])


Напишите функцию `function02`. Функции на вход должен приходить датасет — тензор-матрица признаков объектов. Ваша функция должна создать тензор-вектор с весами (пусть они будут из равномерного распределения на отрезке от 0 до 1) и вернуть их для дальнейшего обучения линейной регрессии без свободного коэффициента. Сделайте эти веса типа `float32`, для них нужно будет в процессе обучения вычислять градиенты (воспользуйтесь `requires_grad`).

Отправляемый файл должен иметь расширение **.py**

In [ ]:
def function02(tensor: torch.tensor) -> torch.Tensor:
    # calculate the number of columns in the tensor
    num_features = tensor.size(1)
    
    # creating the tensor of weights with the same shape as the input tensor и разрешаем вычисление градиента
    weights = torch.rand(num_features, dtype=torch.float32, requires_grad=True)
       
    return weights

Напишите функцию `function03`. Она должна принимать тензор-матрицу с объектами и тензор-вектор с правильными ответами, будем решать задачу регрессии: `def function03(x: torch.Tensor, y: torch.Tensor):`

Создайте внутри функции веса для линейной регрессии (без свободного коэффициента), можете воспользоваться функцией из предыдущего степа. С помощью градиентного спуска подберите оптимальные веса для входных данных (используйте длину шага около `1e-2`). Верните тензор-вектор с оптимальными весами из функции. Ваши обученные веса должны давать **MSE** на обучающей выборке меньше единицы.

Отправляемый файл должен иметь расширение .py

Входные данные будут выглядеть примерно так:

[Input] ---> [(Weights)] ---> [Output]

In [28]:
import torch
import numpy as np

# Задаём количество признаков (фичей) и количество объектов в нашей модели
n_features = 2
n_objects = 300

# Генерируем истинные веса для нашей модели. Они случайны и следуют нормальному (гауссовскому) распределению.
w_true = torch.randn(n_features)

# Создаём матрицу объектов X. Размер матрицы - это (n_objects x n_features). 
# Мы вычитаем 0.5, чтобы центрировать данные вокруг 0, а затем умножаем на 5, чтобы получить более широкий диапазон значений.
# Это делается для того, чтобы сделать данные более "интересными" и сложными для модели.
X = (torch.rand(n_objects, n_features) - 0.5) * 5

# Сгенерируем "истинные" значения y, используя линейное преобразование наших объектов с истинными весами. 
# Затем добавляем нормальный шум, поделенный на два. 
# Это делается для того, чтобы внести некоторое количество случайности и непредсказуемости в наши данные, сделать их более "реалистичными".
Y = X @ w_true + torch.randn(n_objects) / 2



'''
- Инициализируем веса как в прошлом задании
- Пока ошибка больше заданного уровня вычисляем ошибку
- Вычисляем градиенты (обратное распространение)
- Обновляем веса (не забывая отключать подсчет градиентов)
- Обновляем параметры
- Включаем подсчет градиентов
'''

import torch

def function03(x: torch.Tensor, y: torch.Tensor):
    # Количество признаков
    n_features = x.shape[1]
    
    # Инициализируем веса
    w = torch.randn(n_features, requires_grad=True)
    
    # Параметры обучения
    learning_rate = 1e-2
    tolerance = 1e-1
    max_iter = 1000
    loss = torch.tensor(float('inf'))
    
    # Градиентный спуск
    for i in range(max_iter):
        # Вычисляем ошибку
        y_pred = x @ w
        loss = torch.mean((y - y_pred)**2)
        
        if loss.item() < tolerance:
            break

        # Вычисляем градиенты
        loss.backward()
        
        # Отключаем подсчет градиентов для обновления весов
        with torch.no_grad():
            w -= learning_rate * w.grad
        
        # Включаем подсчет градиентов и обнуляем градиенты
        w.grad.zero_()

    return w


In [29]:
weights = function03(X, Y)

In [34]:
weights

tensor([-0.1741,  0.8195], requires_grad=True)

In [32]:
w_true

tensor([-0.1942,  0.7977])

# Lesson 2: Backpropagation

Напишите функцию `create_model`, которая должна возвращать полносвязную нейронную сеть из двух слоев. На вход должно быть 100 чисел, на выход 1, посередине 10. В качестве нелинейности используйте ReLU. Воспользуйтесь `nn.Sequential` и передайте слои как последовательность.

In [37]:
import torch
import torch.nn as nn

def create_model():
    
    # Создание объекта нелинейного слоя
    net = nn.Sequential(
        nn.Linear(100, 10),
        nn.ReLU(),
        nn.Linear(10, 1)
    )
    return net

In [38]:
net = create_model()
net

Sequential(
  (0): Linear(in_features=100, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=1, bias=True)
)

Напишите функцию `train`. Она должна принимать на вход нейронную сеть, даталоадер, оптимизатор и функцию потерь. Она должна иметь следующую сигнатуру: `def train(model: nn.Module, data_loader: DataLoader, optimizer: Optimizer, loss_fn):`

Внутри функции сделайте следующие шаги:

1\. Переведите модель в режим обучения.

2\. Проитерируйтесь по даталоадеру.

3\. На каждой итерации:

    - Занулите градиенты с помощью оптимизатора

    - Сделайте проход вперед (forward pass)

    - Посчитайте ошибку

    - Сделайте проход назад (backward pass)

    - Напечатайте ошибку на текущем батче с точностью до 5 символов после запятой (только число)

    - Сделайте шаг оптимизации

Функция должна вернуть среднюю ошибку за время прохода по даталоадеру.

2 урок > шаг 9

Как перевести модель в режим обучения?

Для этого необходимо использовать метод функцию .train() у функции

Как итерироваться по даталоадеру?

for x, y in data\_loader:

Как занулять градиенты и делать шаг?

Для зануления градиентов можно воспользоваться функцией optimizer.zero\_grad(). Для того чтобы сделать шаг оптимизации можно воспользоваться функцией optimizer.step().

Ответ не принимается в LMS

В LMS сдается скрипт в формате .py, c реализацией функции и со всеми необходимыми импортами. Обязательно проверяем код локально перед сдачей в LMS.

Обратите внимание, что значение loss нужно брать как loss.item() чтобы получить число, а не тензор.

In [39]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import Optimizer

def train(model: nn.Module, data_loader: DataLoader, optimizer: Optimizer, loss_fn):
    model.train()
    total_loss = 0
    for batch in data_loader:
        # Разделите данные на входные данные и целевые значения
        inputs, targets = batch

        # Обнулите градиенты
        optimizer.zero_grad()

        # Проход вперед
        outputs = model(inputs)

        # Вычислите ошибку
        loss = loss_fn(outputs, targets)

        # Проход назад
        loss.backward()

        # Шаг оптимизации
        optimizer.step()

        # Напечатайте ошибку на текущем батче
        print(f'{loss.item():.5f}')

        # Суммируйте ошибку
        total_loss += loss.item()

    # Верните среднюю ошибку
    return total_loss / len(data_loader)


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim
from torch.optim import Optimizer

def train(model: nn.Module, data_loader: DataLoader, optimizer: Optimizer, loss_fn):
    model.train()
    total_loss = 0
    for batch in data_loader:
        # Разделите данные на входные данные и целевые значения
        inputs, targets = batch

        # Обнулите градиенты
        optimizer.zero_grad()

        # Проход вперед
        outputs = model(inputs)

        # Вычислите ошибку
        loss = loss_fn(outputs, targets)

        # Проход назад
        loss.backward()

        # Напечатайте ошибку на текущем батче
        print(f'MSE на шаге: {loss.item():.5f}')
        
        # Шаг оптимизации
        optimizer.step()

        # Суммируйте ошибку
        total_loss += loss.item()

    # Верните среднюю ошибку
    return total_loss / len(data_loader)

Напишите функцию `evaluate`. Она должна принимать на вход нейронную сеть, даталоадер и функцию потерь. Она должна иметь следующую сигнатуру: `def evaluate(model: nn.Module, data_loader: DataLoader, loss_fn):`

Внутри функции сделайте следующие шаги:

1\. Переведите модель в режим инференса (применения)

2\. Проитерируйтесь по даталоадеру

3\. На каждой итерации:

    - Сделайте проход вперед (forward pass)

    - Посчитайте ошибку

Функция должна вернуть среднюю ошибку за время прохода по даталоадеру.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader

def evaluate(model: nn.Module, data_loader: DataLoader, loss_fn):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            # Разделите данные на входные данные и целевые значения
            inputs, targets = batch

            # Проход вперед
            outputs = model(inputs)

            # Вычислите ошибку
            loss = loss_fn(outputs, targets)

            # Суммируйте ошибку
            total_loss += loss.item()

    # Верните среднюю ошибку
    return total_loss / len(data_loader)
